In [ ]:
import gspread
import pandas as pd
from google.oauth2.service_account import Credentials
import json
import requests
from bs4 import BeautifulSoup

In [ ]:
with open('setting.json') as f:
    df = json.load(f)
    SPREADSHEET_KEY = df['KANAGAWA_SPREADSHEET_KEY']
    SECRET_KEY_PATH = df['SECRET_KEY_PATH']
    
scope = ['https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive']
credentials = Credentials.from_service_account_file(
    SECRET_KEY_PATH, scopes=scope)
gc = gspread.authorize(credentials)

In [ ]:
#スプレッドシートキーを指定してワークブックを選択
workbook = gc.open_by_key(SPREADSHEET_KEY).sheet1
shop_name_list = workbook.col_values(1)
shop_url_list = workbook.col_values(2)

#df_shopをcsvに
df_shop = pd.DataFrame([shop_name_list, shop_url_list], index=['name', 'url']).T
df_shift = df_shop.shift(periods=-1,axis=0)
df_shift.to_csv('shop_list.csv')


In [ ]:
#csvfileの読み込み
df_shift = pd.read_csv('shop_list.csv')
df_array = []
for i in range(len(shop_name_list)-1):
    #if i ==3:
        #break
    url = df_shift.at[i, 'url']
#url = df_shop.at[1, 'url']
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    s = soup.find_all(class_ ='w620')
    try:
        tel_url = s[0].find('a').get('href')

    except(AttributeError):
        tel_url = None
        tel = None
        
    else:
        tel_r = requests.get(tel_url)
        tel_soup = BeautifulSoup(tel_r.text)
        tel = tel_soup.find(class_='fs16').text
        

    try:
        other = s[2].text+" "+s[5].text+" "+s[6].text+" "+s[8].text+" "+s[9].text
    
    except(IndexError):
        other = s[2].text+" "+s[5].text+" "+s[6].text+" "+s[8].text
    
    else:
        pass
    
    df = pd.DataFrame({
        'url': [url], 
        '住所' : [s[1].text], 
        '営業時間' :[s[3].text], 
        '定休日':[s[4].text], 
        '設備':[s[6].text], 
        'スタッフ数' :[s[7].text], 
        '備考':[other] , 
        'TEL_URL':[tel_url], 
        'TEL':[tel],
        })
    #df = pd.DataFrame([url, s[1].text, s[3].text, s[4].text, s[6].text, s[7].text, other, tel_url, tel], index = ['url', '住所', '営業時間', '定休日', '設備', 'スタッフ数', '備考', 'TEL_URL', 'TEL'])
    
    df_merge= df_shift.merge(df, on='url')
    df_array.append(df_merge)
df_concat = pd.concat(df_array).reset_index(drop = True)


In [ ]:
df_concat.to_csv('esthetic.csv')